In [1]:
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import Counter, defaultdict
import pandas as pd
import pickle
import numpy as np
from shapely.geometry import Polygon, Point, LinearRing, box
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction import DictVectorizer
%matplotlib notebook

In [2]:
df_train = pd.read_pickle('df_train')
df_test = pd.read_pickle('df_test')
x_train_df = pd.read_pickle('x_train_df')
x_test_df = pd.read_pickle('x_test_df')

In [ ]:
min_x_entry = min(min(df_train.x_entry.describe()['min'], df_test.x_entry.describe()['min']),
                  df_train.x_exit.describe()['min'])
min_y_entry = min(min(df_train.y_entry.describe()['min'], df_test.y_entry.describe()['min']),
                  df_train.y_exit.describe()['min'])
max_x_entry = max(max(df_train.x_entry.describe()['max'], df_test.x_entry.describe()['max']),
                  df_train.x_exit.describe()['max'])
max_y_entry = max(max(df_train.y_entry.describe()['max'], df_test.y_entry.describe()['max']),
                  df_train.y_exit.describe()['max'])
print(min_x_entry, max_x_entry, min_y_entry, max_y_entry)

In [ ]:
def plot_map():
    rect_all = patches.Rectangle((3740998.35481912, -19042656.658487003), (3777099.2656833767 - 3740998.35481912),
                             (- 19382914.9809002 + 19042656.658487003),
                             linewidth=1,edgecolor='g',fill = False,hatch = '\\\\\\', label = 'city')
    city = plt.gca().add_patch(rect_all)
    rect_city = patches.Rectangle((3750901.5068, -19208905.6133), (3770901.5068 - 3750901.5068),
                             (- 19268905.6133 + 19208905.6133),
                             linewidth=1,edgecolor='r',fill = False,hatch = '//////', label = 'city center')
    city_center = plt.gca().add_patch(rect_city)
    plt.legend(handles=[city, city_center])
    plt.grid(True)
#     plt.xlim(min_x_entry, max_x_entry)
    plt.ylim(min_y_entry, max_y_entry)
    plt.xlim(3740000, 3780000)
    plt.ylim(-19000000, - 19400000)
    plt.show()

In [ ]:
def get_box(x,y):
    n1 = -1
    n2 = -1
    if(~np.isnan(x) and ~np.isnan(y)):
        if(x >= 3740000.0000000000 and x < 3745000.0000000000):
            n1 = 1
        elif(x >= 3745000.0000000000 and x < 3750000.0000000000):
            n1 = 2
        elif(x >= 3750000.0000000000 and x < 3755000.0000000000):
            n1 = 3
        elif(x >= 3755000.0000000000 and x < 3760000.0000000000):
            n1 = 4
        elif(x >= 3760000.0000000000 and x < 3765000.0000000000):
            n1 = 5
        elif(x >= 3765000.0000000000 and x < 3770000.0000000000):
            n1 = 6
        elif(x >= 3770000.0000000000 and x < 3775000.0000000000):
            n1 = 7
        else:
            n1 = 8

        if(y >= -19400000.0000000000 and y < -19350000.0000000000):
            n2 = 8
        elif(y >= -19350000.0000000000 and y < -19300000.0000000000):
            n2 = 7
        elif(y >= -19300000.0000000000 and y < -19250000.0000000000):
            n2 = 6
        elif(y >= -19250000.0000000000 and y < -19200000.0000000000):
            n2 = 5
        elif(y >= -19200000.0000000000 and y < -19150000.0000000000):
            n2 = 4
        elif(y >= -19150000.0000000000 and y < -19100000.0000000000):
            n2 = 3
        elif(y >= -19100000.0000000000 and y < -19050000.0000000000):
            n2 = 2
        else:
            n2 = 1
    
        return (n2-1)*8 + n1
    else:
        return 0

In [ ]:
df_test['distance'] = df_test.apply(lambda row: Point(row['x_entry'],row['y_entry']).
                                      distance(Point(row['x_exit'],row['y_exit'])) if ~np.isnan(row['x_exit'])
                                     else np.nan, axis=1)
df_test['velocity'] = df_test.apply(lambda row: (row['distance']/row['duration'])
                                      if (row['duration']!= 0.0 and ~np.isnan(row['x_exit'])) else 0.0, axis=1)
df_train['entry_box'] = df_train.apply(lambda row: get_box(row['x_entry'], row['y_entry']) , axis=1)
df_train['exit_box'] = df_train.apply(lambda row: get_box(row['x_exit'], row['y_exit']) , axis=1)
df_test['entry_box'] = df_test.apply(lambda row: get_box(row['x_entry'], row['y_entry']) , axis=1)
df_test['exit_box'] = df_test.apply(lambda row: get_box(row['x_exit'], row['y_exit']) , axis=1)

In [ ]:
df_train.to_pickle('df_train')
df_test.to_pickle('df_test')

In [ ]:
df = df_train.groupby('hash')
df_list = list(df)

In [ ]:
df_test = df_test.groupby('hash')
df_test_list = list(df_test)

In [ ]:
dataset = []
print(len(df_list))
for index, sample in enumerate(df_list):
    length = sample[1].shape[0]
    count = 0
    features = {}
    features['hash'] = sample[0]
    for index1, row in sample[1].iterrows():
        features['trajectory_id']  = row['trajectory_id']
        if(count < length - 1):
            features['entry_'+ str(row['entry_box'])]  = features.get('entry_'+ str(row['entry_box']),0) + 1
            features['exit_' + str(row['exit_box'])]  = features.get('exit_'+ str(row['exit_box']),0) + 1
        else:
            features['l_entry']  = row['entry_box']
            features['l_exit']  = row['exit_box']
            features['x_entry'] = row['x_entry']
            features['y_entry'] = row['y_entry']
            features['duration'] = row['duration']
            features['city_distance'] = row['city_distance']
            features['hour'] = row['hour']
            features['label'] = row['label']
        count = count + 1
    dataset.append(features)

In [ ]:
test_dataset = []
print(len(df_test_list))
for index, sample in enumerate(df_test_list):
    length = sample[1].shape[0]
    count = 0
    features = {}
    features['hash'] = sample[0]
    for index1, row in sample[1].iterrows():
        features['trajectory_id']  = row['trajectory_id']
        if(count < length - 1):
            features['entry_'+ str(row['entry_box'])]  = features.get('entry_'+ str(row['entry_box']),0) + 1
            features['exit_' + str(row['exit_box'])]  = features.get('exit_'+ str(row['exit_box']),0) + 1
        else:
            features['l_entry']  = row['entry_box']
            features['l_exit']  = 0
            features['x_entry'] = row['x_entry']
            features['y_entry'] = row['y_entry']
            features['duration'] = row['duration']
            features['city_distance'] = row['city_distance']
            features['hour'] = row['hour']
            features['label'] = -1
        count = count + 1
    test_dataset.append(features)

In [ ]:
x_train_df = pd.DataFrame(dataset).fillna(0)

In [ ]:
x_test_df = pd.DataFrame(test_dataset).fillna(0)
x_test_df['entry_5'] = 0.0
x_test_df['exit_5'] = 0.0

In [ ]:
print(x_train_df.shape, x_test_df.shape)

In [ ]:
x_train_df.to_pickle('x_train_df')
x_test_df.to_pickle('x_test_df')

In [3]:
training_dataset = x_train_df.drop(['hash','l_exit','trajectory_id'], axis=1)
testing_dataset = x_test_df.drop(['hash','l_exit','trajectory_id'], axis=1)

In [4]:
x_1 = training_dataset.drop(['label'], axis=1).values
y_1 = training_dataset['label'].values
sc_1 = StandardScaler()
x_1 = sc_1.fit_transform(x_1)

In [5]:
xTrain, xTest, yTrain, yTest = train_test_split(x_1, y_1, test_size = 0.0, random_state = 0)
clf_1 = RandomForestClassifier(n_jobs = -1, n_estimators = 500, max_depth = 60, random_state=42)
# clf_1 = GaussianNB()
# clf_1 = AdaBoostClassifier(n_estimators=100)
# clf_1 = SVC(gamma=2, C=1)
# clf_1 = MLPClassifier(activation= 'relu',hidden_layer_sizes=(10,10,10))
# clf_1 = GaussianProcessClassifier(1.0 * RBF(1.0))
# clf_1 = LogisticRegression(penalty ='l2',dual = True)
# clf_1 = DecisionTreeClassifier(max_depth=1000)
# clf_1 = KNeighborsClassifier(3)
clf_1.fit(xTrain, yTrain)
# yPred = clf_1.predict(xTest)
# print(classification_report(yTest, yPred))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=60, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(xTrain, yTrain)
rf_random.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
# Fit the grid search to the data
grid_search.fit(train_features, train_labels)
grid_search.best_params_
best_grid = grid_search.best_estimator_

In [6]:
x_test_df['label'] =  clf_1.predict(sc_1.fit_transform(testing_dataset.drop(['label'], axis=1).values))

In [7]:
x_test_df['label'] = x_test_df.apply(lambda row: 0 
                                 if ((row['duration'] == 0.0) &
                                     ~(((row['x_entry'] >= 3750901.5068) &
                                                               (row['x_entry'] <= 3770901.5068)) &
                                       ((row['y_entry'] >= -19268905.6133) &
                                        (row['y_entry'] <= -19208905.6133))))
                                 else row['label'], axis=1)
x_test_df['label'] = x_test_df.apply(lambda row: 1
                                 if ((row['duration'] == 0.0) &
                                     (((row['x_entry'] >= 3750901.5068) &
                                                               (row['x_entry'] <= 3770901.5068)) &
                                       ((row['y_entry'] >= -19268905.6133) &
                                        (row['y_entry'] <= -19208905.6133))))
                                 else row['label'], axis=1)

In [8]:
x_test_df.label.value_counts()

0    25249
1     8266
Name: label, dtype: int64

In [9]:
x_test_df.sort_values('trajectory_id',ascending=True)[['trajectory_id','label']].to_csv('output25.csv', index=False)